<a href="https://colab.research.google.com/github/sw6820/swm_prototype/blob/main/oddiya_googleADK_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =======================================================================
#  CELL 1: SETUP - INSTALL LIBRARIES
# =======================================================================
# We now use the official Google Agent Development Kit (ADK).
!pip install -qU google-adk

print("✅ Libraries installed. (Google ADK version)")


✅ Libraries installed. (Google ADK version)


In [ ]:

# =======================================================================
#  CELL 2: IMPORTS & API KEY CONFIGURATION
# =======================================================================
import os
import asyncio
from google.colab import userdata

# Google ADK specific imports
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types as genai_types

# Pydantic is still used for data validation and defining the output structure.
from pydantic import BaseModel, Field
from typing import List

# --- API Key Configuration ---
try:
    # Get Gemini API Key
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    print("✅ Google Gemini API Key configured successfully.")
except Exception as e:
    # CORRECTED: The variable name for the secret was fixed.
    print(f"🚨 SECRET NOT FOUND or other error: {e}. Please ensure 'GOOGLE_API_KEY' is set in Colab Secrets.")


✅ Google Gemini API Key configured successfully.


In [ ]:

# =======================================================================
#  CELL 3: BACKEND - DEFINE DATA STRUCTURES
# =======================================================================
# NOTE: These Pydantic models are the same. They define the required output structure.

class GenerativeActivity(BaseModel):
    name: str = Field(description="The name of the recommended place. The value for this key MUST be in Korean (한국어).")
    description: str = Field(description="A brief description of the activity. The value for this key MUST be in Korean (한국어).")

class DailyGenerativePlan(BaseModel):
    day_number: int = Field(description="The day number for this plan (e.g., 1, 2).")
    city: str = Field(description="The city for this plan. The value for this key MUST be in Korean (한국어).")
    theme_of_the_day: str = Field(description="A fun theme for the day. The value for this key MUST be in Korean (한국어).")
    morning_activity: GenerativeActivity = Field(description="The morning activity. Its content must be in Korean.")
    afternoon_activity: GenerativeActivity = Field(description="The afternoon activity. Its content must be in Korean.")

class FullTripPlan(BaseModel):
    """The complete, structured set of generated recommendations for the trip."""
    plans: List[DailyGenerativePlan]

print("✅ Backend data structures defined.")


✅ Backend data structures defined.


In [ ]:

# =======================================================================
#  CELL 4: BACKEND - AGENT DEFINITION (ADK)
# =======================================================================
# We define an LLM-powered agent to handle travel planning.

# The instruction serves as the system prompt for the agent.
agent_instruction = """
You are a travel planner for families. Your sole task is to generate a JSON object that strictly follows the provided schema. Do not include any other text, explanations, or conversational remarks in your response. Do NOT wrap the JSON in markdown code blocks.

**Critical Rules:**
1. The entire output MUST be a single JSON object that conforms exactly to the following Pydantic schema:
{{
  "plans": [
    {{
      "day_number": 1,
      "city": "City in Korean",
      "theme_of_the_day": "Theme in Korean",
      "morning_activity": {{
        "name": "Activity Name in Korean",
        "description": "Activity Description in Korean"
      }},
      "afternoon_activity": {{
        "name": "Activity Name in Korean",
        "description": "Activity Description in Korean"
      }}
    }},
    // Add more daily plans as needed
  ]
}}
2. The JSON 'keys' (like 'plans', 'day_number', 'city', 'theme_of_the_day', 'morning_activity', 'afternoon_activity', 'name', 'description') MUST be in English.
3. The JSON 'values' (the actual content like the theme, descriptions, and place names) MUST be written in Korean (한국어).

The user will provide the city and the number of days for the trip.
"""

# The output_schema tells the ADK to validate the model's output against our Pydantic model.
# The output_key specifies where to store the validated, structured output in the agent's session state.
travel_agent = LlmAgent(
    name="travel_planner_agent",
    model="gemini-1.5-flash-latest",
    instruction=agent_instruction,
    output_schema=FullTripPlan,
    output_key="trip_plan_data",
)

print("✅ Google ADK Agent defined.")

✅ Google ADK Agent defined.


In [ ]:
# =======================================================================
#  CELL 5: DEFINE INPUT & RUN THE ENGINE (Google ADK Version)
# =======================================================================

async def run_trip_planning():
    # --- Modify your trip input here ---
    trip_plan_input = {
        "서울": 2,
        "부산": 2
    }
    # --------------------------------

    print("===================================================")
    print(" G.O.A.T. Generative Engine (Google ADK Mode) ")
    print("===================================================")
    print(f"\nTravel Plan: {trip_plan_input}")

    all_plans = []
    full_day_counter = 1

    # --- ADK Runner and Session Setup ---
    app_name = "trip_planner_app"
    session_service = InMemorySessionService()
    runner = Runner(
        agent=travel_agent,
        app_name=app_name,
        session_service=session_service
    )

    # Iterate through the user-defined trip plan
    for city, num_days in trip_plan_input.items():
        print(f"\n[Engine] Requesting a {num_days}-day plan for {city} from the ADK Agent...")

        try:
            user_id = "user_colab"
            session_id = f"session_{city.lower()}"

            session = await session_service.create_session(
                app_name=app_name,
                user_id=user_id,
                session_id=session_id
            )

            user_query = f"Please generate a complete {num_days}-day trip itinerary for {city}."
            content = genai_types.Content(role='user', parts=[genai_types.Part(text=user_query)])

            async for event in runner.run_async(
                user_id=user_id,
                session_id=session_id,
                new_message=content
            ):
                pass

            final_session = await session_service.get_session(app_name=app_name, user_id=user_id, session_id=session_id)

            # Retrieve the raw dictionary from the session state
            raw_result_dict = final_session.state.get("trip_plan_data")

            # --- CORRECTION: Manually parse the dictionary into a Pydantic model ---
            # This converts the dict into a FullTripPlan object, which has the .plans attribute.
            if raw_result_dict:
                result = FullTripPlan.model_validate(raw_result_dict)

                # Process and number the days sequentially for the whole trip
                for plan in result.plans:
                    plan.day_number = full_day_counter
                    all_plans.append(plan)
                    full_day_counter += 1
            else:
                 print(f"[Warning] No structured data was returned for {city}.")


        except Exception as e:
            print(f"[Error] An error occurred while generating recommendations for {city}: {e}")

    # --- Display the final generated plan ---
    print("\n\n===================================================")
    print("        ✨ Final Generated Travel Plan ✨")
    print("===================================================")

    if not all_plans:
        print("\nNo plans were generated.")
    else:
        for day_plan in all_plans:
            print("\n---------------------------------------------------")
            print(f"  Day {day_plan.day_number}: '{day_plan.theme_of_the_day}' in {day_plan.city}")
            print("---------------------------------------------------")

            morning = day_plan.morning_activity
            print(f"\n  🌅 Morning: {morning.name}")
            print(f"  👉 {morning.description}")

            afternoon = day_plan.afternoon_activity
            print(f"\n  ☀️ Afternoon: {afternoon.name}")
            print(f"  👉 {afternoon.description}")

    print("\n\n===================================================")
    print("              [Engine Run Complete]")
    print("===================================================")

# Run the asynchronous function
await run_trip_planning()

 G.O.A.T. Generative Engine (Google ADK Mode) 

Travel Plan: {'서울': 2, '부산': 2}

[Engine] Requesting a 2-day plan for 서울 from the ADK Agent...

[Engine] Requesting a 2-day plan for 부산 from the ADK Agent...


        ✨ Final Generated Travel Plan ✨

---------------------------------------------------
  Day 1: '서울의 역사와 문화 체험' in 서울
---------------------------------------------------

  🌅 Morning: 경복궁
  👉 웅장한 경복궁을 방문하여 아름다운 건축물과 역사를 느껴보세요. 왕궁의 위엄과 아름다움에 흠뻑 빠져보세요.

  ☀️ Afternoon: 인사동
  👉 전통적인 한국의 아름다움을 간직한 인사동 거리를 거닐며 다양한 공예품과 전통 차를 즐겨보세요. 한국의 문화를 직접 체험할 수 있는 좋은 기회입니다.

---------------------------------------------------
  Day 2: '서울의 즐거움과 쇼핑' in 서울
---------------------------------------------------

  🌅 Morning: 롯데월드
  👉 짜릿한 놀이기구와 다양한 볼거리로 가득한 롯데월드에서 즐거운 시간을 보내세요. 온 가족이 함께 즐길 수 있는 최고의 놀이공원입니다.

  ☀️ Afternoon: 명동 쇼핑
  👉 명동 거리에서 다양한 쇼핑을 즐기고 맛있는 길거리 음식을 맛보세요. 젊음의 거리에서 활기찬 분위기를 만끽하세요.

---------------------------------------------------
  Day 3: '해변과 문화 체험' in 부산
--------------------------